In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter # type: ignore
import boto3
import os
from azure.storage.blob import BlobServiceClient
from tqdm import tqdm
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.retrievers import BM25Retriever
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain.retrievers import EnsembleRetriever
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnableParallel
import numpy as np
from dotenv  import load_dotenv
load_dotenv()
import shutil

def load_file(file_name):
    loader=[]
    print(file_name.split(".")[-1])
    if file_name.split('.')[-1] == "pptx":
        loader = UnstructuredPowerPointLoader(file_name).load()
    elif file_name.split('.')[-1] == "pdf":
        loader = PyPDFLoader(file_name).load()    
    elif file_name.split('.')[-1] == "docx":
        loader = Docx2txtLoader(file_name).load()
    elif file_name.split('.')[-1] == "html":
        loader = UnstructuredHTMLLoader(file_name).load()
    text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=300,
            chunk_overlap=0,
            is_separator_regex=False,
        )
    pages = text_splitter.split_documents(loader)
    return pages


def file_to_chunks(folder):
    pages=[]
    for file_name in os.listdir(f"{folder}"):
        pages.extend(load_file(f"{folder}\\{file_name}"))
    if folder != "Local_data":
        shutil.rmtree(f"{folder}")
    return pages

def azure_data_download(AZURE_CONNECTION_STRING,CONTAINER_NAME):
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(CONTAINER_NAME)
    if not os.path.exists("Azure_data"):
        os.mkdir("Azure_data")
    for file_name in container_client.list_blobs():
        blob_client = container_client.get_blob_client(file_name)
        with open(f"Azure_data\\{file_name.name}", "wb") as file:
            data = blob_client.download_blob().readall()
            file.write(data)


def aws(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, BUCKET_NAME,object_name):
        # Create an S3 client
    s3 = boto3.client('s3',
                    aws_access_key_id=AWS_ACCESS_KEY_ID,
                    aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

    # List objects in the bucket
    response = s3.list_objects_v2(Bucket=BUCKET_NAME)

    if not os.path.exists("S3_data"):
        os.mkdir("S3_data")

    # Download files in the 'data' object
    for i in response.get('Contents',[]):
        if i['Key'].split('/')[-1] != "" and i['Key'].split('/')[0] == object_name:
            # print(i['Key'])
            file_path = os.path.join("S3_data", i['Key'].split('/')[-1])
            # print(file_path)
            s3.download_file(BUCKET_NAME, i['Key'], file_path)


def generate_queries(query):

    # Multi Query: Different Perspectives
    template = """You are an AI language model assistant. Your task is to generate Four 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines. Original question: {question}"""
    prompt_perspectives = ChatPromptTemplate.from_template(template)


    generate_querie = (
        prompt_perspectives 
        | ChatOpenAI(temperature=0) 
        | StrOutputParser() 
        | (lambda x: x.split("\n"))
        | (lambda x: [query] + x)
    )
    return generate_querie 


def keyword_extractor():
    prompt="""
    You are an AI language model assistant. Your task is to help the user identify key terms in their query.

    Please list the main keywords you want to extract from your query.

    query: {query}
    """
    prompt_perspectives=ChatPromptTemplate.from_template(prompt)
    generate_querie = (
        prompt_perspectives 
        | ChatOpenAI(temperature=0) 
        | StrOutputParser() )
    return generate_querie

def _get(a):
    dd=[]
    for s in a:
        dd.extend(s)
    return dd

def get_unique_documents(doc_list):
    seen_content = set()
    unique_documents = []
    
    for doc in doc_list:
        content = doc.page_content
        if content not in seen_content:
            seen_content.add(content)
            unique_documents.append(doc)
    
    del seen_content
    
    return unique_documents

In [2]:
pages = file_to_chunks("Local_data")

pptx
pdf
pdf
docx
docx
html
docx


In [3]:
pages

[Document(page_content='FREE TEST DATA\n\nPPT FILE\n\n\n\nFREE TEST DATA\n\nLorem ipsum dolor sit amet, consectetur adipiscing elit. Donec iaculis elit nec ante eleifend, eget cursus tortor auctor. Integer at ultrices lorem, eget bibendum turpis. Praesent lacus sapien, ullamcorper.\n\nFTD\n\nFREE TEST DATA\n\n2\n\n\n\nFREE TEST DATA', metadata={'source': 'Local_data\\Free_Test_Data_1MB_PPTX.pptx'}),
 Document(page_content='Nullam nisl ante, pellentesque eu iaculis sit amet, scelerisque et leo. Nunc fringilla tempus odio nec posuere. Vivamus blandit dignissim ante, quis sollicitudin lorem hendrerit id. Interdum et malesuada fames ac ante ipsum primis in faucibus. Nunc enim arcu, bibendum non blandit auctor, tempus et', metadata={'source': 'Local_data\\Free_Test_Data_1MB_PPTX.pptx'}),
 Document(page_content='mi. Vestibulum ligula nisi, feugiat sit amet posuere eget, convallis interdum risus. Donec non blandit dui. In tempor viverra metus ut mollis. Cras cursus, velit eu iaculis pellentes

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
db = FAISS.from_documents(pages, OpenAIEmbeddings())

In [6]:
db.save_local("Local_vectorstore")

In [5]:
Query="What is mlflow?"

In [6]:

faiss_retriever=db.as_retriever(search_kwargs={'k': 10})

Bm25_retriever = BM25Retriever.from_documents(pages)
Bm25_retriever.k = 10


In [7]:
generate_queries(Query).invoke({"question":Query})

['What is mlflow?',
 '1. Can you explain the concept of mlflow?',
 '2. How would you define mlflow?',
 '3. What are the key features of mlflow?',
 '4. In simple terms, what does mlflow refer to?']

In [8]:
map_chain=generate_queries | faiss_retriever.map() | _get | get_unique_documents

In [9]:
key_chain=keyword_extractor() | Bm25_retriever

In [10]:
ensemble_retriever = EnsembleRetriever(
retrievers=[map_chain, key_chain], weights=[0.25, 0.75]
)

In [11]:
model = HuggingFaceCrossEncoder(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2")
compressor = CrossEncoderReranker(model=model, top_n=4)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

final_prompt="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    Question: {question} 
    Context: {context} 
    Answer:"""

final_prompt_perspectives=ChatPromptTemplate.from_template(final_prompt)

c:\Users\shyams\Downloads\RAG with different source\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\shyams\Downloads\RAG with different source\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\shyams\Downloads\RAG with different source\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:

llm_chain2=({"context": itemgetter("query") | compression_retriever,
        "question":itemgetter("query")}
        | final_prompt_perspectives
        | ChatOpenAI(temperature=0) | StrOutputParser() )


In [13]:



        
llm_chain3= ({"context": itemgetter("query") | compression_retriever,
            "question":itemgetter("query")}
            | 
            RunnableParallel({
                "response":  final_prompt_perspectives | ChatOpenAI(temperature=0) | StrOutputParser() ,
                "context": itemgetter("context")
            })
            )

In [14]:
llm_chain3

{
  context: RunnableLambda(itemgetter('query'))
           | ContextualCompressionRetriever(base_compressor=CrossEncoderReranker(model=HuggingFaceCrossEncoder(client=<sentence_transformers.cross_encoder.CrossEncoder.CrossEncoder object at 0x000001C2C85984A0>, model_name='cross-encoder/ms-marco-MiniLM-L-6-v2', model_kwargs={}), top_n=4), base_retriever=EnsembleRetriever(retrievers=[RunnableLambda(generate_queries)
             | RunnableEach(bound=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001C2A9CA5190>, search_kwargs={'k': 10}))
             | RunnableLambda(_get)
             | RunnableLambda(get_unique_documents), ChatPromptTemplate(input_variables=['query'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='\n    You are an AI language model assistant. Your task is to help the user identify key terms in their query.\n\n    Please list the main keywords you 

In [15]:
s=llm_chain3.invoke({"query":Query})

In [16]:
s

{'response': 'MLflow is an open-source platform for the machine learning life cycle, focusing on reproducibility, training, and deployment. It provides facilities for experiment tracking, model management, registry, and deployment interface. MLflow allows practitioners to manage the ML life cycle from model development to deployment in a reliable and scalable environment.',
 'context': [Document(page_content='MLflow and other ecosystem tools\nMLflow is a tool that was created by the open source community to address a gap in open \nsystems for ML systems, focused on reproducibility, model management, and deployment. \nMLflow is by no means a complete tool; it needs other components and is a central part of', metadata={'page': 134, 'source': 'Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf'}),
  Document(page_content='1\nIntroducing MLflow\nMLflow  is an open sour

In [35]:
def meta(s):
    f=[]
    for i in s:
        d=""
        for i,y in i.metadata.items():
            d=d+f"{i} : {y} \n"
        f.append(d)
    return f

In [34]:
meta(s["context"])

['page : 134 \n\nsource : Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf \n\n',
 'page : 19 \n\nsource : Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf \n\n',
 'page : 64 \n\nsource : Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf \n\n',
 'page : 20 \n\nsource : Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf \n\n']

In [21]:
for i in s["context"]:
    print(i.metadata)

{'page': 134, 'source': 'Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf'}
{'page': 19, 'source': 'Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf'}
{'page': 64, 'source': 'Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf'}
{'page': 20, 'source': 'Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf'}


In [28]:
d=""
for i,y in s["context"][0].metadata.items():
    d=d+f"{i} : {y}\n"
print(d)

page : 134
source : Local_data\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf



In [59]:
type(meta(s["context"]))

list

In [56]:
s["context"][0].metadata["page"]

134

In [37]:
meta(s["context"])

['page : 134 \nsource : Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf \n',
 'page : 19 \nsource : Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf \n',
 'page : 64 \nsource : Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf \n',
 'page : 20 \nsource : Local_data\\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf \n']